In [1]:
from random import randint, shuffle
import numpy as np

: 

# Matrice d'adjacence

In [1]:
# Génération de la matrice adjacence à partir d'un fichier
def load_matrix():
    mat = []
    with open('mat_adjacence', 'r') as file:   
        for l in file.readlines():  
            val = [int(x) for x in l.split()]
            mat.append(val)            
    return mat

# Créer une matrice d'adjacence dont le cout du circuit Hamiltonien optimal est celui de la périphérie  
def create_matrix(n_ville):
    mat = np.zeros((n_ville, n_ville))
    
    for i in range(n_ville):
        for j in range(i, n_ville):
            cout = randint(1, 10)
            if i == j:
                cout = 0
            k = i-1 if i-1 > 0 else n_ville
            if (i+1) % n_ville == j or k == j:
                cout = 1
                 
            mat[i][j] = cout
            mat[j][i] = cout

    return mat
 

Initialisation de la population

In [ ]:
def population_init(nb_ville, size):
    population = []
    
    for _ in range(size):
        circuit = list(range(nb_ville))
        shuffle(circuit) # circuit random
        population.append([circuit, 5])
        
    return population



In [ ]:

def empty_mat(width):
    return [[0]*width for _ in range(width)]

class Agent:
    def __init__(self, starting_node, nodes_to_explore, deposit_Q):
        self.starting_node = starting_node # Toute les fourmies commencent au même endroit
        self.deposit_Q = deposit_Q
        self.reset_exploration(nodes_to_explore)
    
    def choose_next_location(self, pheromone_mat):
        # TODO : On choisit la prochaine ville a explorer aléatoirement selon les phéromones,
        # avec une proba non nul qu'on explore une ville jamais exploré par les agents
        return self.unexplored[randint(0, len(self.unexplored))]
    
    def reset_exploration(self, nodes_to_explore):
        self.unexplored = list(nodes_to_explore)
        self.unexplored.remove(self.starting_node)
        self.explored = [self.starting_node]
    
    def get_travel_length(self, mat_cout):
        travel_length = 0
        for i in range(1, len(self.explored)):
            depart = self.explored(i-1)
            arrival = self.explored(i) 
            
            travel_length += mat_cout[depart][arrival]
            
        return travel_length
    
    def deposite_pheromone(self, mat_cost, pheromone_mat):
        
        pheromone_quantity = self.deposit_Q /  self.get_travel_length(mat_cost)
        
        
        for i in range(1, len(self.explored)):
            depart = self.explored(i-1)
            arrival = self.explored(i) 
            
            # On dépose les phéromones sur les arrettes
            pheromone_mat[depart][arrival] += pheromone_quantity
    
    def advance(self, mat_cost, pheromone_mat): # avance l'agent
        
        # Si on a atteint toutes les villes 
        if len(self.unexplored) < 0:
            # Une fois la tournée terminé, l'agent dépose les phéromones
            self.deposite_pheromone(mat_cost, pheromone_mat)
            # On reset les villes explorer et on recommence l'exploration 
            self.reset_exploration(self.explored)
         
        # On choisit la prochaine destination à explorer
        curr_node = self.explored.pop(randint(0, len(self.unexplored)))
        self.explored.append(curr_node)
        
        
def evaporation(pheromone_mat, disipation_rate):
    for i in range(pheromone_mat):
        for j in range(pheromone_mat[0]):
            if pheromone_mat[i][j] > 0:
                pheromone_mat[i][j] -= disipation_rate

In [ ]:
def main():
    size = 5
    epoch_max = 100
    agents_number = 5
    starting_node = 0
    disipation_rate = 1
    
    mat_cost = create_matrix(size)
    pheromone_mat = empty_mat(size)
    agents = [Agent(starting_node, list(range(size))) for _ in range(agents_number)]
    
    disipation_rate = 1
    
    for i in range(epoch_max):
        # On avance chaque agents dans leurs exploration
        for agent in agents:
            agent.advance(mat_cost, pheromone_mat)

        evaporation(pheromone_mat, disipation_rate)